# 🧾 TaxAlly - Real-Time Tax & Compliance Copilot

**AI-powered tax assistant for Indian individuals and micro-businesses**

This notebook demonstrates the TaxAlly agent - a multi-tool AI copilot that:
- Understands your tax profile
- Tracks compliance deadlines
- Analyzes transactions
- Identifies compliance risks
- Provides actionable guidance

---

## 1️⃣ Setup & Installation

In [ ]:
# Install required packages
!pip install -q requests gradio

In [ ]:
# Set your API key (get free key from console.groq.com)
import os
from google.colab import userdata

# Option 1: From Colab secrets
try:
    os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')
except:
    # Option 2: Paste directly (not recommended for shared notebooks)
    os.environ['GROQ_API_KEY'] = 'your-api-key-here'

print("✅ API key configured")

## 2️⃣ Core Agent Implementation

In [ ]:
# === AGENT CORE ===

from abc import ABC, abstractmethod
from dataclasses import dataclass, field
from enum import Enum
from typing import Any, Optional, Callable
from datetime import datetime, timedelta
import json
import re
import requests


class AgentMode(Enum):
    INDIVIDUAL = "individual"
    MICRO_BUSINESS = "micro_business"


@dataclass
class AgentContext:
    user_id: str
    session_id: str
    mode: AgentMode
    entity_id: Optional[str] = None
    timestamp: datetime = field(default_factory=datetime.utcnow)


@dataclass
class ToolCall:
    tool_name: str
    parameters: dict
    reasoning: str


@dataclass
class ToolResult:
    success: bool
    data: Any
    error: Optional[str] = None


class BaseTool(ABC):
    @property
    @abstractmethod
    def name(self) -> str:
        pass

    @property
    @abstractmethod
    def description(self) -> str:
        pass

    @property
    def category(self) -> str:
        return "general"

    @abstractmethod
    def execute(self, params: dict, context: AgentContext) -> Any:
        pass


print("✅ Agent core loaded")

In [ ]:
# === TOOLS IMPLEMENTATION ===

class ComplianceRuleEngine(BaseTool):
    """Check compliance status against Indian tax rules."""
    
    THRESHOLDS = {
        "gst_registration": 2000000,  # 20 lakh
        "advance_tax": 10000,
    }
    
    @property
    def name(self) -> str:
        return "compliance_check"
    
    @property
    def description(self) -> str:
        return "Check GST, Income Tax, TDS compliance status and identify risks"
    
    def execute(self, params: dict, context: AgentContext) -> dict:
        profile = params.get("profile", {})
        turnover = params.get("turnover", 0)
        income = params.get("income", 0)
        
        result = {
            "gst": self._check_gst(profile, turnover),
            "income_tax": self._check_income_tax(income),
            "advance_tax": self._check_advance_tax(income)
        }
        return result
    
    def _check_gst(self, profile: dict, turnover: float) -> dict:
        threshold = self.THRESHOLDS["gst_registration"]
        is_registered = profile.get("gst_registered", False)
        
        if turnover > threshold and not is_registered:
            return {
                "status": "non_compliant",
                "risk": f"GST registration required. Turnover ₹{turnover:,.0f} exceeds ₹{threshold:,.0f}",
                "action": "Apply for GST registration immediately"
            }
        return {"status": "compliant", "message": "GST status OK"}
    
    def _check_income_tax(self, income: float) -> dict:
        if income > 300000:
            tax = self._calculate_tax(income)
            return {
                "filing_required": True,
                "estimated_tax": tax,
                "regime_suggestion": "new_regime" if income < 1500000 else "evaluate_both"
            }
        return {"filing_required": False}
    
    def _check_advance_tax(self, income: float) -> dict:
        tax = self._calculate_tax(income)
        if tax > self.THRESHOLDS["advance_tax"]:
            return {
                "required": True,
                "estimated_annual_tax": tax,
                "next_deadline": self._get_next_advance_tax_deadline()
            }
        return {"required": False}
    
    def _calculate_tax(self, income: float) -> float:
        """New regime FY 2024-25."""
        if income <= 300000: return 0
        elif income <= 700000: return (income - 300000) * 0.05
        elif income <= 1000000: return 20000 + (income - 700000) * 0.10
        elif income <= 1200000: return 50000 + (income - 1000000) * 0.15
        elif income <= 1500000: return 80000 + (income - 1200000) * 0.20
        else: return 140000 + (income - 1500000) * 0.30
    
    def _get_next_advance_tax_deadline(self) -> str:
        today = datetime.now()
        deadlines = [
            (6, 15, "Q1 - 15%"),
            (9, 15, "Q2 - 45%"),
            (12, 15, "Q3 - 75%"),
            (3, 15, "Q4 - 100%")
        ]
        for month, day, label in deadlines:
            d = today.replace(month=month, day=day)
            if d > today:
                return f"{d.strftime('%B %d')} ({label})"
        return "March 15 next year (Q4 - 100%)"


class CalendarTracker(BaseTool):
    """Track compliance deadlines."""
    
    DEADLINES = {
        "gstr1": {"day": 11, "desc": "GSTR-1 (Outward supplies)"},
        "gstr3b": {"day": 20, "desc": "GSTR-3B (Summary return)"},
        "tds": {"day": 7, "desc": "TDS payment"},
    }
    
    @property
    def name(self) -> str:
        return "calendar"
    
    @property
    def description(self) -> str:
        return "Get upcoming tax compliance deadlines"
    
    def execute(self, params: dict, context: AgentContext) -> list:
        days_ahead = params.get("days_ahead", 30)
        gst_registered = params.get("gst_registered", False)
        
        today = datetime.now()
        upcoming = []
        
        for key, deadline in self.DEADLINES.items():
            if key.startswith("gst") and not gst_registered:
                continue
                
            day = deadline["day"]
            next_date = today.replace(day=min(day, 28))
            if next_date <= today:
                if today.month == 12:
                    next_date = today.replace(year=today.year+1, month=1, day=day)
                else:
                    next_date = today.replace(month=today.month+1, day=day)
            
            days_until = (next_date - today).days
            if days_until <= days_ahead:
                upcoming.append({
                    "deadline": key,
                    "description": deadline["desc"],
                    "date": next_date.strftime("%Y-%m-%d"),
                    "days_until": days_until,
                    "urgency": "🔴" if days_until <= 3 else "🟡" if days_until <= 7 else "🟢"
                })
        
        return sorted(upcoming, key=lambda x: x["days_until"])


class TransactionAnalyzer(BaseTool):
    """Categorize transactions."""
    
    PATTERNS = {
        "salary": ["salary", "payroll", "wages"],
        "professional": ["consulting", "freelance", "professional"],
        "rent_received": ["rent received", "rental"],
        "interest": ["interest", "fd", "savings"],
        "investment": ["mutual fund", "mf", "sip", "stocks"],
    }
    
    @property
    def name(self) -> str:
        return "categorize"
    
    @property
    def description(self) -> str:
        return "Categorize financial transactions for tax purposes"
    
    def execute(self, params: dict, context: AgentContext) -> dict:
        description = params.get("description", "").lower()
        
        for category, patterns in self.PATTERNS.items():
            if any(p in description for p in patterns):
                return {
                    "category": category,
                    "confidence": 0.85,
                    "tax_treatment": self._get_tax_treatment(category)
                }
        
        return {"category": "uncategorized", "confidence": 0, "needs_review": True}
    
    def _get_tax_treatment(self, category: str) -> str:
        treatments = {
            "salary": "Taxable under 'Salaries'. Check Form 16 for TDS.",
            "professional": "Taxable under 'PGBP'. Consider presumptive taxation (44ADA).",
            "rent_received": "Taxable under 'House Property'. 30% standard deduction available.",
            "interest": "Taxable under 'Other Sources'. Check for TDS u/s 194A.",
            "investment": "Check holding period for STCG/LTCG treatment."
        }
        return treatments.get(category, "Review needed")


print("✅ Tools loaded")

In [ ]:
# === LLM PROVIDER ===

class GroqLLM:
    """Groq API for fast inference."""
    
    def __init__(self, model: str = "llama-3.1-70b-versatile"):
        self.api_key = os.getenv("GROQ_API_KEY")
        self.model = model
        self.base_url = "https://api.groq.com/openai/v1/chat/completions"
    
    def generate(self, system_prompt: str, user_message: str, context: dict = None) -> str:
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "Content-Type": "application/json"
        }
        
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_message}
        ]
        
        payload = {
            "model": self.model,
            "messages": messages,
            "temperature": 0.7,
            "max_tokens": 1024
        }
        
        response = requests.post(self.base_url, headers=headers, json=payload)
        
        if response.status_code == 200:
            return response.json()["choices"][0]["message"]["content"]
        else:
            return f"Error: {response.status_code} - {response.text}"


print("✅ LLM provider loaded")

In [ ]:
# === TAXALLY AGENT ===

class TaxAllyAgent:
    """Main agent orchestrating tools and LLM."""
    
    def __init__(self):
        self.llm = GroqLLM()
        self.tools = {
            "compliance_check": ComplianceRuleEngine(),
            "calendar": CalendarTracker(),
            "categorize": TransactionAnalyzer()
        }
        self.user_profile = {}  # In-memory state for demo
        self.conversation = []
    
    def _build_system_prompt(self) -> str:
        profile_str = json.dumps(self.user_profile, indent=2) if self.user_profile else "Not yet collected"
        
        return f"""You are TaxAlly, an AI tax compliance assistant for India.

USER PROFILE:
{profile_str}

AVAILABLE TOOLS:
- compliance_check: Check GST/Income Tax/TDS compliance. Params: profile, turnover, income
- calendar: Get upcoming deadlines. Params: days_ahead, gst_registered
- categorize: Categorize a transaction. Params: description

TO USE A TOOL, respond with:
```tool
{{"tool": "tool_name", "params": {{...}}}}
```

GUIDELINES:
1. Ask clarifying questions if needed
2. Use tools to get accurate data
3. Explain your reasoning
4. Flag compliance risks clearly
5. Provide actionable next steps

DISCLAIMER: You provide guidance, not legal/tax advice. Complex cases need a CA.
"""
    
    def _parse_tool_calls(self, response: str) -> list:
        """Extract tool calls from LLM response."""
        tool_calls = []
        pattern = r"```tool\n(.*?)\n```"
        matches = re.findall(pattern, response, re.DOTALL)
        
        for match in matches:
            try:
                data = json.loads(match)
                tool_calls.append(data)
            except json.JSONDecodeError:
                continue
        
        return tool_calls
    
    def _execute_tools(self, tool_calls: list) -> str:
        """Execute tool calls and format results."""
        results = []
        context = AgentContext(
            user_id="demo_user",
            session_id="demo_session",
            mode=AgentMode.INDIVIDUAL
        )
        
        for call in tool_calls:
            tool_name = call.get("tool")
            params = call.get("params", {})
            
            if tool_name in self.tools:
                result = self.tools[tool_name].execute(params, context)
                results.append(f"[{tool_name}]: {json.dumps(result, indent=2)}")
            else:
                results.append(f"[{tool_name}]: Unknown tool")
        
        return "\n\n".join(results)
    
    def chat(self, user_message: str) -> str:
        """Main chat interface."""
        # Check for profile updates in message
        self._extract_profile_updates(user_message)
        
        # Generate initial response
        system_prompt = self._build_system_prompt()
        response = self.llm.generate(system_prompt, user_message)
        
        # Check for tool calls
        tool_calls = self._parse_tool_calls(response)
        
        if tool_calls:
            # Execute tools
            tool_results = self._execute_tools(tool_calls)
            
            # Get final response with tool results
            followup = f"""{user_message}

TOOL RESULTS:
{tool_results}

Based on these results, provide a clear response to the user."""
            
            response = self.llm.generate(system_prompt, followup)
        
        # Store conversation
        self.conversation.append({"user": user_message, "assistant": response})
        
        return response
    
    def _extract_profile_updates(self, message: str):
        """Extract profile info from user message."""
        message_lower = message.lower()
        
        # Simple pattern matching for demo
        if "freelancer" in message_lower or "freelance" in message_lower:
            self.user_profile["entity_type"] = "individual"
            self.user_profile["income_source"] = "profession"
        
        if "business" in message_lower:
            self.user_profile["entity_type"] = "proprietorship"
        
        # Extract turnover if mentioned
        turnover_match = re.search(r"(?:turnover|revenue|income)[^0-9]*(\d+(?:,\d+)*(?:\.\d+)?)[^0-9]*(?:lakh|lac|L)?", message_lower)
        if turnover_match:
            amount = float(turnover_match.group(1).replace(",", ""))
            if "lakh" in message_lower or "lac" in message_lower:
                amount *= 100000
            elif "crore" in message_lower:
                amount *= 10000000
            self.user_profile["turnover"] = amount
        
        # GST status
        if "gst registered" in message_lower:
            self.user_profile["gst_registered"] = True
        elif "not gst" in message_lower or "no gst" in message_lower:
            self.user_profile["gst_registered"] = False


# Initialize agent
agent = TaxAllyAgent()
print("✅ TaxAlly Agent ready!")

## 3️⃣ Interactive Demo

In [ ]:
# Test the agent with sample queries

print("=" * 60)
print("TaxAlly Demo - Sample Conversations")
print("=" * 60)

queries = [
    "I'm a freelance software developer with annual income of 15 lakh. What are my tax obligations?",
    "What are my upcoming compliance deadlines?",
    "I received 50,000 as consulting fees. How should I categorize this?"
]

for query in queries:
    print(f"\n👤 User: {query}")
    print("-" * 40)
    response = agent.chat(query)
    print(f"🤖 TaxAlly: {response}")
    print("=" * 60)

In [ ]:
# Interactive chat

def interactive_chat():
    print("\n" + "="*60)
    print("💬 TaxAlly Interactive Chat")
    print("Type 'quit' to exit")
    print("="*60 + "\n")
    
    while True:
        user_input = input("👤 You: ")
        if user_input.lower() in ['quit', 'exit', 'q']:
            print("\n👋 Goodbye! Remember to file your returns on time!")
            break
        
        response = agent.chat(user_input)
        print(f"\n🤖 TaxAlly: {response}\n")

# Uncomment to run interactive chat
# interactive_chat()

## 4️⃣ Gradio Web Interface

In [ ]:
import gradio as gr

def respond(message, history):
    response = agent.chat(message)
    return response

demo = gr.ChatInterface(
    fn=respond,
    title="🧾 TaxAlly - Tax Compliance Copilot",
    description="AI-powered tax assistant for Indian individuals and micro-businesses",
    examples=[
        "I'm a freelancer earning 12 lakh per year. What are my tax obligations?",
        "Do I need to register for GST?",
        "What are my upcoming tax deadlines?",
        "How do I categorize my consulting income?",
        "Should I pay advance tax?"
    ],
    theme="soft"
)

demo.launch(share=True)

## 5️⃣ API Server (Optional)

In [ ]:
# Simple API endpoint using Gradio

def api_chat(message: str, profile: str = "{}") -> str:
    """API endpoint for chat."""
    try:
        profile_dict = json.loads(profile)
        agent.user_profile.update(profile_dict)
    except:
        pass
    
    return agent.chat(message)

api = gr.Interface(
    fn=api_chat,
    inputs=[
        gr.Textbox(label="Message"),
        gr.Textbox(label="Profile JSON (optional)", value="{}")
    ],
    outputs=gr.Textbox(label="Response"),
    title="TaxAlly API"
)

# api.launch(share=True)  # Uncomment to run API

---

## Architecture Notes

### Current MVP
- Single LLM (Groq/Llama 3.1)
- 3 core tools (compliance, calendar, categorizer)
- In-memory state
- Gradio interface

### Future Ready
- Tool interface allows easy addition of new tools
- LLM provider abstraction supports model switching
- State schema supports multi-entity, multi-user
- Agent modes for different use cases

### Next Steps
1. Add document parsing (Form 16, Form 26AS)
2. Integrate bank statement analysis
3. Add notification system
4. Connect to government APIs
5. Build mobile app